In [1]:
# Import utility.py and db_utility
import scripts.utils as ut
from scripts.db_utils import get_db_engine, open_connection, close_connection, get_table_count
from sqlalchemy import text

# Load mapped characteristics from file
path = ut.os.path.join(ut.repo_root, 'data', 'processed', 'sql', 'characteristics_mapped.csv')
df = ut.read_generic_csv(path, ',')

print(df.head())
print(f"✅ Loaded {len(df)} characteristics")

                                              descar  category
0         apartment_(1_bedroom)_in_city_centre_(usd)         1
1      apartment_(1_bedroom)_outside_of_centre_(usd)         1
2        apartment_(3_bedrooms)_in_city_centre_(usd)         1
3     apartment_(3_bedrooms)_outside_of_centre_(usd)         1
4  price_per_square_meter_to_buy_apartment_in_cit...         1
✅ Loaded 44 characteristics


In [2]:
# DB connection
engine = get_db_engine()
conn = open_connection(engine)

inserted, skipped = 0, 0

# -- Insert into regcar
# Get current max idncar increment manually
result = conn.execute(text("SELECT MAX(idcara) FROM regcar;"))
start_id = (result.scalar() or 0) + 1

print(f"ℹ️ Starting manual ID insert from idncar = {start_id}")

for i, row in df.iterrows():
    # Dont want the header
    if row['descar'].strip().lower() == 'descar':
        continue
    
    descar = row['descar'].strip().lower()
    idncat = int(row['category'])
    idcara = start_id + i

    exists = conn.execute(
        text("SELECT 1 FROM regcar WHERE LOWER(TRIM(descar)) = :descar"),
        {"descar": descar}
    ).fetchone()

    if not exists:
        conn.execute(
            text("""
                INSERT INTO regcar (idcara, descar, idncat)
                VALUES (:idcara, :descar, :idncat)
            """),
            {
                "idcara": idcara,
                "descar": row["descar"],
                "idncat": idncat,
            },
        )
        inserted += 1
    else:
        skipped += 1


print(f"✅ regcar population completed: {inserted} inserted, {skipped} skipped.")
print(f"📊 Total rows in regcar: {get_table_count(conn, 'regcar')}")

# Close connection
conn.commit()
close_connection(conn)

🔍 Loading .env
🔧 Variables loaded: user=user, host=localhost, db=livingcost
ℹ️ Starting manual ID insert from idncar = 45
✅ regcar population completed: 0 inserted, 44 skipped.
📊 Total rows in regcar: 44
✅ DB connection closed


In [3]:
# -- VERIFICATION
# DB connection
engine = get_db_engine()
conn = open_connection(engine)

# Verification
result = conn.execute(text("SELECT * FROM regcar LIMIT 10;"))
print("\n🔎 Sample of inserted data:")

for row in result:
    print(row)

# Close Connection
close_connection(conn)

🔍 Loading .env
🔧 Variables loaded: user=user, host=localhost, db=livingcost

🔎 Sample of inserted data:
(1, 'apartment_(1_bedroom)_in_city_centre_(usd)', 1)
(2, 'apartment_(1_bedroom)_outside_of_centre_(usd)', 1)
(3, 'apartment_(3_bedrooms)_in_city_centre_(usd)', 1)
(4, 'apartment_(3_bedrooms)_outside_of_centre_(usd)', 1)
(5, 'price_per_square_meter_to_buy_apartment_in_city_centre_(usd)', 1)
(6, 'price_per_square_meter_to_buy_apartment_outside_of_centre_(usd)', 1)
(7, 'basic_(electricity,_heating,_cooling,_water,_garbage)_for_85m2_apartment_(usd)', 2)
(8, 'internet_(60_mbps_or_more,_unlimited_data,_cable/adsl)_(usd)', 2)
(9, 'one-way_ticket_(local_transport)_(usd)', 3)
(10, 'monthly_pass_(regular_price)_(usd)', 3)
✅ DB connection closed
